## CHAPTER 10
# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 10.1 How to Think About Group Operations

In [2]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)
df

,key1,key2,data1,data2
0,a,1,-2.164690,0.946590
1,a,2,-0.821775,-0.134352
2,None,1,-0.730250,0.691866
3,b,2,0.662584,-0.395107
4,b,1,0.058994,1.518270
5,a,<NA>,0.081849,-1.922607
6,None,1,0.462282,0.662356


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a   -0.968206
b    0.360789
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -2.164690
      2      -0.821775
b     1       0.058994
      2       0.662584
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,-2.164690,-0.821775
b,0.058994,0.662584


In [7]:
state = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = np.array([2005, 2005, 2006, 2005, 2006, 2005, 2006])
df["data1"].groupby([state, years]).mean()

CA  2005   -0.369963
    2006   -0.730250
OH  2005   -0.751053
    2006    0.260638
Name: data1, dtype: float64

In [8]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.968206,-0.370123
b,1.5,0.360789,0.561582


In [9]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.593416,0.954771
2,-0.079596,-0.264729


In [10]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -2.164690  0.946590
     2    -0.821775 -0.134352
b    1     0.058994  1.518270
     2     0.662584 -0.395107

In [11]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [12]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [13]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [14]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [15]:
for name , group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -2.164690  0.946590
1    a     2 -0.821775 -0.134352
5    a  <NA>  0.081849 -1.922607
b
  key1  key2     data1     data2
3    b     2  0.662584 -0.395107
4    b     1  0.058994  1.518270


In [16]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2    data1    data2
0    a     1 -2.16469  0.94659
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2 -0.821775 -0.134352
('b', np.int64(1))
  key1  key2     data1    data2
4    b     1  0.058994  1.51827
('b', np.int64(2))
  key1  key2     data1     data2
3    b     2  0.662584 -0.395107


In [17]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.662584,-0.395107
4,b,1,0.058994,1.518270


In [18]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -2.164690  0.946590
 1    a     2 -0.821775 -0.134352
 5    a  <NA>  0.081849 -1.922607,
 'b':   key1  key2     data1     data2
 3    b     2  0.662584 -0.395107
 4    b     1  0.058994  1.518270}

In [19]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -2.164690  0.946590
1 -0.821775 -0.134352
2 -0.730250  0.691866
3  0.662584 -0.395107
4  0.058994  1.518270
5  0.081849 -1.922607
6  0.462282  0.662356
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\695880334.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


### Selecting a Column or Subset of Columns

In [20]:
df

,key1,key2,data1,data2
0,a,1,-2.164690,0.946590
1,a,2,-0.821775,-0.134352
2,None,1,-0.730250,0.691866
3,b,2,0.662584,-0.395107
4,b,1,0.058994,1.518270
5,a,<NA>,0.081849,-1.922607
6,None,1,0.462282,0.662356


In [21]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [22]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [23]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.946590
     2    -0.134352
b    1     1.518270
     2    -0.395107

In [24]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     1       0.946590
      2      -0.134352
b     1       1.518270
      2      -0.395107
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [26]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.986363,1.446720,0.463969,0.982927,1.794162
Steve,-2.341916,0.072348,-0.319274,0.532935,0.622384
Wanda,-0.415881,NaN,NaN,-1.361603,0.491156
Jill,-0.978812,-1.784197,1.596357,1.472226,1.844148
Trey,-0.683352,-0.773266,0.646499,0.312429,-1.263727


In [27]:
mapping = {"a": "red", "b": "red", "c": "blue",
            "d": "blue", "e": "red", "f" : "orange"}


In [28]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,1.446896,4.227245
Steve,0.213662,-1.647184
Wanda,-1.361603,0.075275
Jill,3.068584,-0.918861
Trey,0.958927,-2.720345


In [29]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2727320543.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [31]:
people

,a,b,c,d,e
Joe,0.986363,1.446720,0.463969,0.982927,1.794162
Steve,-2.341916,0.072348,-0.319274,0.532935,0.622384
Wanda,-0.415881,NaN,NaN,-1.361603,0.491156
Jill,-0.978812,-1.784197,1.596357,1.472226,1.844148
Trey,-0.683352,-0.773266,0.646499,0.312429,-1.263727


In [32]:
people.groupby(len).sum()  # group by length of the index value

,a,b,c,d,e
3,0.986363,1.446720,0.463969,0.982927,1.794162
4,-1.662164,-2.557462,2.242856,1.784655,0.580421
5,-2.757798,0.072348,-0.319274,-0.828667,1.113540


In [33]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.986363,1.446720,0.463969,0.982927,1.794162
4,two,-0.978812,-1.784197,0.646499,0.312429,-1.263727
5,one,-2.341916,0.072348,-0.319274,-1.361603,0.491156


### Grouping by Index Levels

In [34]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -2.426264 -0.730629  1.109786  0.871261 -0.573532
1      1.374130 -0.897427 -0.158023  0.315782  0.887315
2      1.333555 -1.619342 -1.298383 -0.785320 -0.062581
3     -0.266410 -0.534133  0.112385 -1.092677  0.527717

In [35]:
hier_df.groupby(level="cty", axis="columns").count()

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [36]:
df

,key1,key2,data1,data2
0,a,1,-2.164690,0.946590
1,a,2,-0.821775,-0.134352
2,None,1,-0.730250,0.691866
3,b,2,0.662584,-0.395107
4,b,1,0.058994,1.518270
5,a,<NA>,0.081849,-1.922607
6,None,1,0.462282,0.662356


In [37]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -2.164690
      1   -0.821775
b     4    0.058994
      3    0.662584
Name: data1, dtype: float64

In [38]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.246539,2.869197
b,1,0.603590,1.913377


In [39]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.968206  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.360789  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a    -0.369963  0.081849   3.0 -0.370123  1.449056 -1.922607 -1.028479   
b     0.511687  0.662584   2.0  0.561582  1.352962 -0.395107  0.083237   

                                   
           50%       75%      max  
key1                               
a    -0.134352  0.406119  0.94659  
b     0.561582  1.039926  1.51827  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [40]:
# tips = sns.load_dataset("tips")
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [41]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [42]:
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [43]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [44]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [45]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [46]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [47]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\4198102554.py:2: FutureWarning: The provided callable <function var at 0x00000298C1D8DEE0> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [48]:
grouped.agg({"tip": np.max, "size": "sum"})

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3453032214.py:1: FutureWarning: The provided callable <function max at 0x00000298C1D8D260> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [49]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [50]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [51]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True) # Added numeric_only=True to avoid FutureWarning # শুধু numeric কলাম নাও

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine

In [52]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [53]:
tips.groupby("smoker").apply(top)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2530541573.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [54]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\643835440.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [55]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [56]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [57]:
def f(group):
    return group.describe()
grouped.apply(f)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3509319160.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(f)


total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

In [58]:
tips.groupby("smoker", group_keys=False).apply(top)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\3851957478.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile and Bucket Analysis

In [59]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.061705,0.839211
1,-0.170663,0.228827
2,0.892758,0.357295
3,1.352651,-0.042824
4,1.333485,-1.284246


In [60]:
quartiles = pd.cut(frame["data1"], 4)
quartiles[:10]

0    (-1.371, 0.202]
1    (-1.371, 0.202]
2     (0.202, 1.774]
3     (0.202, 1.774]
4     (0.202, 1.774]
5    (-1.371, 0.202]
6     (0.202, 1.774]
7    (-1.371, 0.202]
8    (-1.371, 0.202]
9    (-1.371, 0.202]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.95, -1.371] < (-1.371, 0.202] < (0.202, 1.774] < (1.774, 3.347]]

In [61]:
def get_stats(group):
    return pd.DataFrame(
        {
            "min": group.min(),
            "max": group.max(),
            "count": group.count(),
            "mean": group.mean(),
            "std": group.std(),
        }
    )
grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

C:\Users\sahor\AppData\Local\Temp\ipykernel_14300\2176047469.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean       std
data1                                                               
(-2.95, -1.371] data1 -2.943453 -1.375453     90 -1.775198  0.337189
                data2 -2.954705  1.936076     90 -0.052205  1.023451
(-1.371, 0.202] data1 -1.368631  0.198690    492 -0.494208  0.428886
                data2 -2.863982  3.240705    492  0.018969  0.987486
(0.202, 1.774]  data1  0.205830  1.754817    379  0.814309  0.421457
                data2 -3.173200  3.012310    379 -0.024359  0.986567
(1.774, 3.347]  data1  1.820200  3.346871     39  2.123965  0.297756
                data2 -2.043690  2.685919     39 -0.008447  1.088847

In [62]:
grouped.agg(['min', 'max', 'count', 'mean', 'std'])

data1                                         data2  \
                      min       max count      mean       std       min   
data1                                                                     
(-2.95, -1.371] -2.943453 -1.375453    90 -1.775198  0.337189 -2.954705   
(-1.371, 0.202] -1.368631  0.198690   492 -0.494208  0.428886 -2.863982   
(0.202, 1.774]   0.205830  1.754817   379  0.814309  0.421457 -3.173200   
(1.774, 3.347]   1.820200  3.346871    39  2.123965  0.297756 -2.043690   

                                                     
                      max count      mean       std  
data1                                                
(-2.95, -1.371]  1.936076    90 -0.052205  1.023451  
(-1.371, 0.202]  3.240705   492  0.018969  0.987486  
(0.202, 1.774]   3.012310   379 -0.024359  0.986567  
(1.774, 3.347]   2.685919    39 -0.008447  1.088847

In [63]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp[:10]

0    1
1    1
2    3
3    3
4    3
5    1
6    3
7    1
8    0
9    1
Name: data1, dtype: int64

In [64]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean       std
data1                                                     
0     data1 -2.943453 -0.696059    250 -1.282888  0.449824
      data2 -2.954705  3.240705    250  0.011685  1.046948
1     data1 -0.695024 -0.020855    250 -0.355638  0.200070
      data2 -2.863982  2.636244    250 -0.016184  0.981882
2     data1 -0.019379  0.651818    250  0.315694  0.199156
      data2 -3.173200  2.369175    250 -0.007079  0.901477
3     data1  0.653243  3.346871    250  1.276991  0.474647
      data2 -2.773286  3.012310    250 -0.008132  1.041620

### Example: Filling Missing Values with Group-Specific Values

In [65]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    0.695839
2         NaN
3    0.432887
4         NaN
5    0.617089
dtype: float64

In [66]:
s.fillna(s.mean())

0    0.581938
1    0.695839
2    0.581938
3    0.432887
4    0.581938
5    0.617089
dtype: float64

In [67]:
state = [
    "Ohio",
    "New York",
    "Vermont",
    "Florida",
    "Oregon",
    "Nevada",
    "California",
    "Idaho",
]

In [68]:
group_key = ["East"] * 4 + ["West"] * 4
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [69]:
data = pd.Series(np.random.standard_normal(8), index=state)
data

Ohio          0.964188
New York     -1.978369
Vermont      -0.469830
Florida      -1.349021
Oregon        0.111196
Nevada       -0.877421
California    0.058801
Idaho        -0.134617
dtype: float64

In [70]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.964188
New York     -1.978369
Vermont            NaN
Florida      -1.349021
Oregon        0.111196
Nevada             NaN
California    0.058801
Idaho              NaN
dtype: float64

In [72]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [73]:
data.groupby(group_key).mean()

East   -0.787734
West    0.084999
dtype: float64

In [74]:
def fill_mean(group):
    return group.fillna(group.mean())

In [75]:
data.groupby(group_key).apply(fill_mean)

East  Ohio          0.964188
      New York     -1.978369
      Vermont      -0.787734
      Florida      -1.349021
West  Oregon        0.111196
      Nevada        0.084999
      California    0.058801
      Idaho         0.084999
dtype: float64

In [76]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [77]:
data.groupby(group_key).apply(fill_func)

East  Ohio          0.964188
      New York     -1.978369
      Vermont       0.500000
      Florida      -1.349021
West  Oregon        0.111196
      Nevada       -1.000000
      California    0.058801
      Idaho        -1.000000
dtype: float64

In [79]:
data

Ohio          0.964188
New York     -1.978369
Vermont            NaN
Florida      -1.349021
Oregon        0.111196
Nevada             NaN
California    0.058801
Idaho              NaN
dtype: float64